Chapitre 10 - Session utilisations et Insertions SQL
===

## Bibliographie

- [Formulaires avec WTForms](https://blog.miguelgrinberg.com/post/the-flask-mega-tutorial-part-iii-web-forms)
- [How NOT to Store Passwords](https://www.youtube.com/watch?v=8ZtInClXe1Q)

In [1]:
from IPython.display import HTML
HTML('<iframe width="560" height="315" src="https://www.youtube-nocookie.com/embed/8ZtInClXe1Q" frameborder="0" allow="autoplay; encrypted-media" allowfullscreen></iframe>')

## Créer des comptes

La création de compte est une des tâches les plus courantes dans le cadre d'un développement d'application. C'est aussi une des plus dangereuses : vous êtes responsables de la sécurité de vos utilisateurs et de leurs données. On apprendra donc ici à sécuriser un maximum nos données sensibles.

### Gérer des mots de passe

Pour enregister un mot de passe, on va *a minima* encrypter ce mot de passe. On utilise d'ailleurs à mauvais escient ce terme car ce que l'on va calculer réellement est un hash. Contrairement à l'encryptage, le hash n'est pas reversible. Si quelque chose est hashé en "qqewretet", la seule manière de trouver l'original est de "brute-forcer" ce dernier, c'est à dire de générer tous les mots de passe possibles (ce qui est, disons le, plutôt difficile).

L'autre manière de casser un Hash, c'est d'avoir la malchance d'avoir l'algorithme de hashage de cassé. On en trouve trois actuellement recommandés (*cf.* *[
About Secure Password Hashing](http://security.blogoverflow.com/2013/09/about-secure-password-hashing/)*) :
- PBKDF2
- bcrypt
- scrypt

Une personne a déjà produit [un plugin](https://bitbucket.org/mhallin/py-scrypt/src) pour utiliser `scrypt`. Installons le :

In [3]:
# Afin d'éviter un aller-retour dans l'environnement virtuel :
!pip install scrypt

Le plugin `scrypt` possède une fonction simple `encrypt(sel, motdepasse, maxtime=0.1)` où maxtime est un décimal représentant le temps d'encryptage maximum :

In [19]:
import scrypt
print(scrypt.hash("motdepasse", "un petit sel en passant par là"))

b"\xe1\x08\xfd\xdeO\xae\xd4s_\xf1\x08\xa0\x1fq\xb1C\xcaK<\xfbB\xd2\x08\xd2:\xa8\xb4\xf1q\xa1\xe9\x96,.\xdbg&\xfd\xa9\xaa\xee\xe1'.q\xc97\xf4\x02A\xf0\x9fr\xfd+{h\x86\xa3[*o\x85\xe1"


Nous avons ici une chaîne en `bytes` : elle repose sur un encodage différent de l'encodage UTF-8 de `str()`. Pour des raisons propres à l'utf-8, il est fort probable que le côté aléatoire de `scrypt` rende le hash de mot de passe intraduisible en UTF-8.

#### Constante de "SEL" (*salt*)

Pour notre application, nous allons intégrer un nouveau module pour stocker nos constantes. Cela permettra de gérer les configurations générales plus proprement : créons donc `gazetteer/constantes.py` où nous stockons ce sel.

In [24]:
%pycat cours-flask/exemple16/gazetteer/constantes.py

Le sel devant être unique et non-devinable, on rajoute un avertissement au cas où la personne mettant ce site en fonctionnement n'avait pas connaissance de ce changement de configuration.

#### Méthode propre

On va maintenant créer une fonction pour vérifier une connexion. Rappel de notre classe `User`:

```python
class User(db.Model):
    user_id = db.Column(db.Integer, unique=True, nullable=False, primary_key=True, autoincrement=True)
    user_nom = db.Column(db.Text, nullable=False)
    user_login = db.Column(db.String(45), nullable=False, unique=True)
    user_email = db.Column(db.Text, nullable=False)
    user_password = db.Column(db.String(64), nullable=False)
```

Pour vérifier la validité de de l'identification de l'utilisateur, on va retrouver l'utilisateur, comparer le hash généré du mot de passe avec celui enregistré :

```python
def identification(login, motdepasse):
    """ Identifie un utilisateur. Si cela fonctionne, renvoie les données de l'utilisateurs.
    
    :param login: Login de l'utilisateur
    :param motdepasse: Mot de passe envoyé par l'utilisateur
    :returns: Si réussite, données de l'utilisateur. Sinon None
    :rtype: User or None
    """
    utilisateur = User.query.filter(User.user_login == login)
    if motdepasse == scrypt.hash(motdepasse, SEL):
        return utilisateur
    return None
```

Pour que cette fonction soit facile à retrouver, on va l'enregistrer sous la responsabilité de la classe `User` :

```python
class User(db.Model):
    user_id = db.Column(db.Integer, unique=True, nullable=False, primary_key=True, autoincrement=True)
    user_nom = db.Column(db.Text, nullable=False)
    user_login = db.Column(db.String(45), nullable=False, unique=True)
    user_email = db.Column(db.Text, nullable=False)
    user_password = db.Column(db.String(64), nullable=False)
    
    @staticmethod
    def identification(login, motdepasse):
        """ Identifie un utilisateur. Si cela fonctionne, renvoie les données de l'utilisateurs.

        :param login: Login de l'utilisateur
        :param motdepasse: Mot de passe envoyé par l'utilisateur
        :returns: Si réussite, données de l'utilisateur. Sinon None
        :rtype: User or None
        """
        utilisateur = User.query.filter(User.user_login == login)
        if motdepasse == scrypt.hash(motdepasse, SEL):
            return utilisateur
        return None
```

On précède notre fonction de `@staticmethod` pour mettre d'appeler cette fonction ainsi :

```python
utilisateur = User.identification(login, motdepasse)
```

#### Premier insert

On va maintenant effectuer notre premier insert ! Et oui, il va falloir créer des comptes. Pour créer un compte, il nous faudra aussi une fonction de création de compte.

Pour créer un enregistrement ou une mise à jour, MySQL et SQLAlchemy fonctionne par session de changement, un peu comme git : on crée un ensemble de modification, on les stocke pour l'envoi (`db.session.add(lachoseaenvoyer)`) et on envoie à MySQL (`db.session.commit()`). Prenez le temps de bien lire le contenu qui suit :

```python
class User(db.model):
    # ...
    @staticmethod
    def creer(login, email, nom, motdepasse):
        """ Crée un compte utilisateur-rice. Retourne un tuple (booléen, User ou liste).
        Si il y a une erreur, la fonction renvoie False suivi d'une liste d'erreur
        Sinon, elle renvoie True suivi de la donnée enregistrée

        :param login: Login de l'utilisateur-rice
        :param email: Email de l'utilisateur-rice
        :param nom: Nom de l'utilisateur-rice
        :param motdepasse: Mot de passe de l'utilisateur-rice (Minimum 6 caractères)

        """
        erreurs = []
        if not login:
            erreurs.append("Le login fourni est vide")
        if not email:
            erreurs.append("L'email fourni est vide")
        if not nom:
            erreurs.append("Le nom fourni est vide")
        if not motdepasse or len(motdepasse) < 6:
            erreurs.append("Le mot de passe fourni est vide ou trop court")

        # On vérifie que personne n'a utilisé cet email ou ce login
        uniques = User.query.filter(
            db.or_(User.user_email == email, User.user_login == login)
        ).count()
        if uniques > 0:
            erreurs.append("L'email ou le login sont déjà inscrits dans notre base de données")

        # Si on a au moins une erreur
        if len(erreurs) > 0:
            return False, erreurs

        # On crée un utilisateur
        utilisateur = User(
            user_nom=nom,
            user_login=login,
            user_email=email,
            user_password=scrypt.hash(motdepasse, SEL)
        )

        try:
            # On l'ajoute au transport vers la base de données
            db.session.add(utilisateur)
            # On envoie le paquet
            db.session.commit()

            # On renvoie l'utilisateur
            return True, utilisateur
        except Exception as erreur:
            return False, [str(erreur)]
```

On ajoute ensuite la route pour s'inscrire (ainsi qu'un lien dans le menu !) :

```python

```
